### Introducción a la Investigación Operativa y la Optimización

### • Clase 5  - Métodos Cuasi-Newton

**Nazareno Faillace Mullen - Departamento de Matemática, FCEN, UBA**

La idea, como en el caso del Método de Newton, es aproximar $f$ mediante una expresión cuadrática. Sin embargo, en los métodos cuasi-Newton, se utiliza, en vez de $Hf(x)$, una matriz $B_k$ que sea simétrica definida positiva. La idea es que $B_k$ aproxime a $Hf(x_k)$.

Como hemos visto, en el método de Newton la dirección de descenso (si $Hf(x_k)\succ 0$) viene dada por:

$$-Hf(x_k)^{-1}\nabla f(x^{k})$$

Dos enfoques: <br>
• Aproximar $Hf(x_k)$ con matrices $B_k$ <br>
• Aproximar $Hf(x_k)^{-1}$ con matrices $H_k$ [$\leftarrow$ trabajamos con este]

### Algoritmo general de un método Cuasi-Newton

Dados: $f,\; x_0 \in \mathbb{R}^n, \; H_0\in \mathbb{R}^{n\times n}\;\;\text{definida positiva},\; \varepsilon > 0, \; k_{MAX} > 0$ <br>
$k = 0$ <br>
REPETIR  mientras $\lVert \nabla f(x_k) \rVert > \varepsilon$ y $k<k_{MAX}$:<br>
&nbsp;&nbsp;&nbsp;&nbsp; Definir $d_k = -H_k\nabla f(x_k)$<br>
&nbsp;&nbsp;&nbsp;&nbsp; Obtener $t_k$ (sección áurea, Armijo o Wolfe)<br>
&nbsp;&nbsp;&nbsp;&nbsp; Hacer $x_{k+1} = x_k + t_kd_k$ <br>
&nbsp;&nbsp;&nbsp;&nbsp; Determinar $H_{k+1}$ definida positiva<br>
&nbsp;&nbsp;&nbsp;&nbsp; $k=k+1$<br>
DEVOLVER $x_k$

__Obs:__ si $H_k = I \rightarrow$ Método del Gradiente <br>
si $H_k = Hf(x_k)^{-1} \rightarrow$ Método de Newton

__Obs:__ como $H_0$ podemos tomar la identidad, ya que es definida positiva. También podría tomarse $Hf(x_0)$ si es definida positiva.

In [ ]:
def gradiente(f, x):
    return np.array([f[i](x) for i in range(len(f))]).reshape(-1,1)

def wolfe(A, x, d, c1 = 0.5, c2 = 0.75):
  alfa = 0
  t = 1
  beta = np.inf
  while True:
    if f(x+t*d,A) > f(x,A) + c1*t*np.dot((A@x).T,d):
      beta = t
      t = 1/2 * (alfa + beta)
    elif np.dot((A@(x+t*d)).T,d) < c2*np.dot((A@x).T,d):
      alfa = t
      if beta == np.inf:
        t = 2*alfa
      else:
        t = 1/2 * (alfa + beta)
    else:
      break
  return t


def derivada_parcial(f,x,i):
    h = 0.1
    e_i = np.zeros(len(x))
    e_i[i] = 1
    z = (f(x + h*e_i) - f(x - h*e_i)) / (2*h)
    h = h/2
    y = (f(x + h*e_i) - f(x - h*e_i)) / (2*h)
    error = np.linalg.norm(y-z)
    eps = 1e-8
    while error>eps and (y != np.nan) and (y != np.inf):
        error = np.linalg.norm(y-z)
        z = y
        h = h/2
        y = (f(x + h*e_i) - f(x - h*e_i)) / (2*h)
    return z

def gradiente(f, x):
    return np.array([derivada_parcial(f, x, i) for i in range(len(x))])

def broyden_mala(f, x0, H0, eps=1e-4, max_iter=1000):
  k = 0
  H_k = H0
  d0 = -H0 @ gradiente(f(x0, A), x0)
  t0 = wolfe(f, x0, d0)
  x_k_prev = x0
  x_k = x_k_prev - t_k * d_k
  s_k = x_k - x_k_prev
  gradiente_xk = gradiente(f(x_k, A), x_k)
  gradiente_xk_prev = gradiente(f(x_k_prev, A), x_k_prev)
  y_k = gradiente_xk - gradiente_xk_prev
  while np.linalg.norm(gradiente_xk) > eps and k < max_iter:
    d_k = -H_k @ gradiente_xk
    s_k = x_k - x_k_prev
    y_k = gradiente_xk - gradiente_xk_prev
    t_k = wolfe(A, x_k, d_k)
    x_k = x_k + t_k * d_k
    H_k = H_k + (s_k - H_k * y_k)*(s_k - H_k * y_k).T/(y_k.T * (s_k - H_k*y_k))
    k += 1
  return x_k, k


In [ ]:
def dfp(f, x0, H0, eps=1e-4, max_iter=1000):
  k = 0
  H_k = H0
  d0 = -H0 @ gradiente(f(x0, A), x0)
  t0 = wolfe(f, x0, d0)
  x_k_prev = x0
  x_k = x_k_prev - t_k * d_k
  s_k = x_k - x_k_prev
  gradiente_xk = gradiente(f(x_k, A), x_k)
  gradiente_xk_prev = gradiente(f(x_k_prev, A), x_k_prev)
  y_k = gradiente_xk - gradiente_xk_prev
  while np.linalg.norm(gradiente_xk) > eps and k < max_iter:
    d_k = -H_k @ gradiente_xk
    s_k = x_k - x_k_prev
    y_k = gradiente_xk - gradiente_xk_prev
    t_k = wolfe(A, x_k, d_k)
    x_k = x_k + t_k * d_k
    H_k = H_k + (s_k * s_k.T)/(y_k.T * s_k) - (H_k * y_k * y_k.T * H_k)/(y_k.T * H_k * y_k)
    k += 1
  return x_k, k

In [ ]:
def bfgs(f, x0, H0, eps=1e-4, max_iter=1000):
  k = 0
  H_k = H0
  d0 = -H0 @ gradiente(f(x0, A), x0)
  t0 = wolfe(f, x0, d0)
  x_k_prev = x0
  x_k = x_k_prev - t_k * d_k
  s_k = x_k - x_k_prev
  gradiente_xk = gradiente(f(x_k, A), x_k)
  gradiente_xk_prev = gradiente(f(x_k_prev, A), x_k_prev)
  y_k = gradiente_xk - gradiente
  while np.linalg.norm(gradiente_xk) > eps and k < max_iter:
    d_k = -H_k @ gradiente_xk
    s_k = x_k - x_k_prev
    y_k = gradiente_xk - gradiente_xk_prev
    t_k = wolfe(A, x_k, d_k)
    x_k = x_k + t_k * d_k
    H_k = H_k + (1 + (y_k.T * H_k * y_k)/(s_k.T * y_k)) * (s_k * s_k.T)/(s_k.T * y_k) - (s_k * y_k.T * H_k + H_k * y_k * s_k.T)/(s_k.T * y_k)
    k += 1
  return x_k, k

Sean $y_k = \nabla f(x_{k+1}) - \nabla f(x_k)$ y $s_k = x_{k+1} - x_k$, una propiedad que debe satisfacer $H_{k+1}$ definida por el algoritmo es que:
$$H_{k+1}y_j = s_j \quad \forall j=0,1,\dots,k$$

### Broyden (Mala)

$$H_{k+1} = H_k + \dfrac{(s_k-H_ky_k)(s_k-H_ky_k)^T}{y_k^T(s_k-H_ky_k)}$$

### Método DFP (Davindon, Fletcher y Powell)

$$H_{k+1} = H_k + \dfrac{s_k(s_k)^T}{(y_k)^Ts_k} - \dfrac{H_ky_k(y_k)^TH_k}{(y_k)^TH_ky_k}$$

### Método BFGS (Broyden, Fletcher, Goldfarb y Shanno)

$$H_{k+1} = H_k + \left(1 + \dfrac{(y_k)^TH_ky_k}{(s_k)^Ty_k}\right)\dfrac{s_k(s_k)^T}{(s_k)^Ty_k} - \dfrac{s_k(y_k)^TH_k + H_ky_k(s_k)^T}{(s_k)^Ty_k} $$

### Importante

Como en general se trabaja con vectores columna:<br>
• $\mathbf{u^T}\mathbf{v} = <\mathbf{u}, \mathbf{v}>\rightarrow$ `u @ v` <br>

• $ {\displaystyle \mathbf{u} \mathbf{v^T} = \mathbf {u} \otimes \mathbf {v} ={\begin{bmatrix}u_{1}v_{1}&u_{1}v_{2}&\dots &u_{1}v_{n}\\u_{2}v_{1}&u_{2}v_{2}&\dots &u_{2}v_{n}\\\vdots &\vdots &\ddots &\vdots \\u_{m}v_{1}&u_{m}v_{2}&\dots &u_{m}v_{n}\end{bmatrix}}}\rightarrow$ `np.outer(u, v)`

### Método de Barzilai-Borwein-Raydan (BBR)

Sean $y_k = \nabla f(x_{k+1}) - \nabla f(x_k)$ y $s_k = x_{k+1} - x_k$.

Dados: $f,\; x^0 \in \mathbb{R}^n, \; \varepsilon > 0, \; k_{MAX} > 0$ <br>
Definir $d_0 = -\nabla f(x_k)$ <br>
Obtener $t_0$ (sección áurea, Armijo o Wolfe) <br>
$x_1 = x_0 + t_0* d_0$ <br>
$k = 1$<br>
REPETIR  mientras $\lVert \nabla f(x_k) \rVert > \varepsilon$ y $k<k_{MAX}$:<br>

&nbsp;&nbsp;&nbsp;&nbsp; Definir $d_k = -\nabla f(x_k)$<br>
&nbsp;&nbsp;&nbsp;&nbsp; Definir $t_k = \dfrac{s_{k-1}^Ts_{k-1}}{s_{k-1}^Ty_{k-1}}$ ( si $f$ es cuadrática, $t_k = \dfrac{s_{k-1}^T s_{k-1}}{s_{k-1} ^T A s_{k-1}}$) <br>
&nbsp;&nbsp;&nbsp;&nbsp; Hacer $x_{k+1} = x_k + t_kd_k$ <br>
&nbsp;&nbsp;&nbsp;&nbsp; $k=k+1$<br>
DEVOLVER $x_k$


## Ejercicios

1. Implementar los tres métodos de Cuasi-Newton para el caso de **funciones cuadráticas** $f(x)=\frac{1}{2}x^T A x + bx + c$ con $A\succ 0$. Debe tomar como input la matriz $A$, el vector $b$, el vector inicial $x_0$ y la cantidad máxima de iteraciones $k_{MAX}$. Utilizar que para este tipo de funciones sabemos que: <br>
• $\nabla f(x_k) = Ax_k +b$ <br>
• $t_k = -\dfrac{(Ax_k +b)^T d_k}{(d_k)^TAd_k}$ <br>
• $y_k=\nabla f(x_{k+1}) - \nabla f(x_k)=(Ax_{k+1} + b) - (Ax_k+b) = A(x_{k+1} - x_k)= As_k$
2. Testear los cuatro métodos con las funciones cuadráticas dadas por $f(x)=\frac{1}{2}x^T A_i x$ para cada una de las $A_i$ que figuran debajo. Comparar el número de iteraciones de cada uno con el de Gradiente Conjugado.
3. Para $A_4$ graficar el recorrido que DFP y el recorrido de Gradiente Conjugado.
4. Para $A_4$, comparar el recorrido realizado por el Método del Gradiente y el recorrido de BBR. **Utilizar $x_0 = (-0.15, 1)$** como punto inicial.
5. Implementar los cuatro métodos para funciones en general y aplicarlos para hallar el minimizador de la función de resta de exponenciales. Comparar con Gradiente Conjugado. Se pueden elegir, por ejemplo, estos puntos iniciales: <br>
• $x_0=(1,0)$ <br>
• $x_0 =(2, 1.5)$ <br>
• $x_0=(0.5, 0.5)$ <br>
• $x_0=(0,0)$ <br>

In [ ]:
import numpy as np
from numpy.linalg import norm, eigvals, solve
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

A1 = np.array([[8, 3, 3, 6, 5, 4, 4, 3, 6, 3],
             [3, 4, 2, 2, 2, 1, 3, 3, 3, 2],
             [3, 2, 5, 2, 1, 2, 4, 2, 4, 1],
             [6, 2, 2, 6, 3, 2, 4, 2, 4, 2],
             [5, 2, 1, 3, 5, 4, 1, 2, 4, 3],
             [4, 1, 2, 2, 4, 5, 1, 2, 5, 2],
             [4, 3, 4, 4, 1, 1, 6, 2, 4, 2],
             [3, 3, 2, 2, 2, 2, 2, 4, 4, 2],
             [6, 3, 4, 4, 4, 5, 4, 4, 8, 3],
             [3, 2, 1, 2, 3, 2, 2, 2, 3, 4]])

A2 = np.array([[2, 0, 1, 0, 1],
               [0, 2, 1, 1, 1],
               [1, 1, 3, 1, 1],
               [0, 1, 1, 1, 0],
               [1, 1, 1, 0, 2]])

A3 = np.diag(np.random.randint(1, 5, 100))

A4 = np.array([[10, 0], [0, 1]])

In [ ]:
def resta_exponenciales(x):
    s1 = np.exp(-x[0] ** 2 - x[1] ** 2)
    s2 = np.exp(-(x[0] - 1) ** 2 - (x[1] - 1) ** 2)
    return (s1 - s2) * 2

def a_forma_cuadratica(A):
    """
    Transforma la función dada por (1/2)(x^T A x) a una función dada en términos de x1 y x2 para que sea posible
    graficar sus curvas de nivel en R2.
    A tiene que ser una matriz de 2x2
    """
    def forma_cuadratica(x):
        return 0.5*(A[0,0]*(x[0]**2) + (A[0,1]+A[1,0])*x[0]*x[1] + A[1,1]*(x[1]**2))
    return forma_cuadratica

def plot_fun(f, limites, points=None):
    """
    f : función a graficar
    limites : toma una tupla (x1,x2,y1,y2) de los límites del gráfico: grafica en el dominio [x1,x2] x [y1,y2]
    points : lista de puntos a graficar sobre la superficie; se ingresa como una lista de tuplas (x,y,z)
    """
    init_notebook_mode(connected=True)

    x = np.linspace(limites[0], limites[1], 1000)
    y = np.linspace(limites[2], limites[3], 1000)
    X, Y = np.meshgrid(x, y)
    Z = f((X, Y))
    data = [go.Surface(x=x, y=y, z=Z)]
    if points is not None:
        for p in points:
            data.append(go.Scatter3d(x=[p[0]], y=[p[1]], z=[p[2]], mode='markers'))
    fig = go.Figure(data=data)
    iplot(fig)

%matplotlib notebook
def graficar_recorrido(f, limites, recorrido=None, levels=10):
    """
    Función que grafica curvas de nivel y, opcionalmente, el recorrido de un método.
    f : es la función a graficar (tiene que ir de R2 en R)
    limites : es una lista o tupla de números: [a,b,c,d]. Va a graficar la función en el cuadrado [a,b] x [c,d]
    recorrido : acepta una lista de arrays bidimensionales para graficar el recorrido de un método
    levels : cantidad de curvas de nivel a graficar
    """
    plt.figure()
    x = np.linspace(limites[0], limites[1], 1000)
    y = np.linspace(limites[2], limites[3], 1000)
    X, Y = np.meshgrid(x, y)
    Z = f((X, Y))
    plt.contour(X,Y,Z, cmap='plasma', levels=levels)
    if recorrido is not None:
        x_coords = [x[0] for x in recorrido]
        y_coords = [x[1] for x in recorrido]
        plt.plot(x_coords, y_coords, marker='o', lw=2, ms=8)
    plt.tight_layout()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

In [ ]:
# EJEMPLO PARA GRAFICAR EL RECORRIDO EN EL EJERCICIO 3 y 4
x_opt, iteraciones, recorrido = metodo_gradiente(A4, np.zeros(2), np.ones(2), 100)
f = a_forma_cuadratica(A4)
graficar_recorrido(f, [-1, 1, -1, 1], recorrido)